In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime



In [ ]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    if 'likeCount' in response_video_stats['items'][0]['statistics']:
      like_count = response_video_stats['items'][0]['statistics']['likeCount']
    else:
      like_count = 0
    
    if 'commentCount' in response_video_stats['items'][0]['statistics']:
      comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    else:
      comment_count = 0

    
    url_video_time = 'https://www.googleapis.com/youtube/v3/videos?id='+ video_id +'&part=contentDetails&key='+API_KEY
    response_video_time = requests.get(url_video_time).json()
    if 'duration' in response_video_time['items'][0]['contentDetails']:
      duration = response_video_time['items'][0]['contentDetails']['duration']
    else:
      duration = 0

    url_video_describe = 'https://www.googleapis.com/youtube/v3/videos?part=snippet&id='+video_id+'&key='+ API_KEY
    response_video_describe = requests.get(url_video_describe).json()
    if 'tags' in response_video_describe['items'][0]['snippet']:
      tags = response_video_describe['items'][0]['snippet']['tags']
    else:
      tags =[]
    
    if 'description'  in response_video_describe['items'][0]['snippet']:
      description = response_video_describe['items'][0]['snippet']['description']
    else:
      description = ''
    return view_count, like_count, comment_count, duration, tags, description

In [ ]:
def get_videos(df):
    pageToken = ""
    while 1:
        sub_url = 'https://www.googleapis.com/youtube/v3/channels?part=statistics&id=' + CHANNEL_ID +'&key='+ API_KEY
        if 'subscriberCount' in requests.get(sub_url).json()['items'][0]['statistics']:
          subscriber = requests.get(sub_url).json()['items'][0]['statistics']['subscriberCount']
        else:
          subscriber = 0
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        

        response = requests.get(url).json()
        ct = datetime.now()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count, duration_count, tags, description= get_video_details(video_id)

                df = df.append({"datetime":ct ,'video_id':video_id,'video_title':video_title,'video_tags':tags,
                                'video_description' : description,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count, "subscriber_count":subscriber,
                                "comment_count":comment_count, 'duration_count' : duration_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
def get_videos_list(df):
    pageToken = ""
    while 1:
        url = 'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet%2C+contentDetails&maxResults=50&playlistId='+LIST_ID+'&key='+API_KEY

        response = requests.get(url).json()
        CHANNEL_ID = response['items'][0]['snippet']['videoOwnerChannelId']
        sub_url = 'https://www.googleapis.com/youtube/v3/channels?part=statistics&id=' + CHANNEL_ID +'&key='+ API_KEY +pageToken
        if 'subscriberCount' in requests.get(sub_url).json()['items'][0]['statistics']:
          subscriber = requests.get(sub_url).json()['items'][0]['statistics']['subscriberCount']
        else:
          subscriber = 0
        
        ct = datetime.now()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['snippet']['resourceId'] == "youtube#video":
                video_id = video['snippet']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({"datetime":ct ,'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count, "subscriber_count":subscriber,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
df2 = pd.DataFrame(columns=["datetime","video_id","video_title",'video_tags',
                                'video_description' ,"upload_date","view_count","like_count","subscriber_count","comment_count", 'duration_count']) 


In [ ]:
#API_KEY = enter your own api key
#1362
CHANNEL_ID_list = ['UC93EEyk9ZHkB4b4RX-mn9WA','UCRp--eWwsLI_uIkCnsbfwFQ','UCn0JHLJWiO3_Y5xfNPcF_Ug','UCKU0iO0axnimO3b0G-BOO_g','UC8nmojd8biOW3rErBHH4cbg','UCT41vlFeZ_asAUCY7BOiJVQ','UCmK4gbkHU_afGKdB9RARcqA','UC1dPSiB4cK10mdchOcKrK8g']

for CHANNEL_ID in CHANNEL_ID_list:
  df2 = get_videos(df2)







In [ ]:
df2

,datetime,video_id,video_title,video_tags,video_description,upload_date,view_count,like_count,subscriber_count,comment_count,duration_count


In [ ]:
df2.to_csv('youtube.csv')